In [1]:
from vdw_evolve import get_data_as_pd, extract_structure, get_cell_from_structure_file  # vdw.parser
from vdw_evolve import AnnealingSolver, MechanicSolver, GeneticSolver  # vdw.solvers
from vdw_evolve import allign_along_10  # vdw.solvers_utils

## Download data:

In [2]:
# select db file
database_path = "../database/c2db-20211702.db"
structure_path = "../database/2Dmaterials"


In [3]:
# Choose the selection criteria
options = 'is_magnetic=True, thermodynamic_stability_level=3'
props = ["formula", "spgnum", "spacegroup", "uid", "asr_id"]
raw_df = get_data_as_pd(database_path, options, props)
raw_df = raw_df[(raw_df["spacegroup"] != 'P1') & (raw_df["spacegroup"] != 'Pc') & (raw_df["spacegroup"] != 'P-1')]

# Print
print("The number of elements is:", len(list(raw_df)))
print(raw_df[["formula", "uid"]])

rows <generator object Database.select at 0x7ff472183820>
The number of elements is: 5
       formula                     uid
0       Mn2Se2     Mn2Se2-a1d595c25fc5
1        Ru2F8      Ru2F8-5b1d25d726e0
2         V2F8       V2F8-6d78fbe605b3
3     Bi2MnTe4   MnBi2Te4-a8bb2517b2b7
4      CrGa2S4    CrGa2S4-e6aae0f0d940
..         ...                     ...
394     Cr2Cl6     Cr2Cl6-9f9e75488d50
395     Pt2Cl6     Pt2Cl6-20b975a3a1e8
396    V2Br2N2    Br2N2V2-186b2da4920f
397  Mn2Cl2Se2  Cl2Mn2Se2-6bdadd86e6c4
400       VSSe       SSeV-0f3ba3ea815f

[337 rows x 2 columns]


In [4]:
# Download (only first 10 for the sake of example)
extract_structure(raw_df["uid"][:10], save_path=structure_path)


https://cmrdb.fysik.dtu.dk/c2db/row/Mn2Se2-a1d595c25fc5/data/structure.json
file:  ../database/2Dmaterials/Mn2Se2-a1d595c25fc5.json found
https://cmrdb.fysik.dtu.dk/c2db/row/Ru2F8-5b1d25d726e0/data/structure.json
file:  ../database/2Dmaterials/Ru2F8-5b1d25d726e0.json found
https://cmrdb.fysik.dtu.dk/c2db/row/V2F8-6d78fbe605b3/data/structure.json
file:  ../database/2Dmaterials/V2F8-6d78fbe605b3.json found
https://cmrdb.fysik.dtu.dk/c2db/row/MnBi2Te4-a8bb2517b2b7/data/structure.json
file:  ../database/2Dmaterials/MnBi2Te4-a8bb2517b2b7.json found
https://cmrdb.fysik.dtu.dk/c2db/row/CrGa2S4-e6aae0f0d940/data/structure.json
file:  ../database/2Dmaterials/CrGa2S4-e6aae0f0d940.json found
https://cmrdb.fysik.dtu.dk/c2db/row/F2N3V4-3f5b11d43a1b/data/structure.json
file:  ../database/2Dmaterials/F2N3V4-3f5b11d43a1b.json found
https://cmrdb.fysik.dtu.dk/c2db/row/N3Sc4-daf440ed0646/data/structure.json
file:  ../database/2Dmaterials/N3Sc4-daf440ed0646.json found
https://cmrdb.fysik.dtu.dk/c2db/row/

## Parsing 

In [5]:
# Choose the two structures that will be overlapped
uid = [raw_df["uid"][0], raw_df["uid"][3]]  # "Pd2Se4-12f02221b8c5", "C2-a6735a4a3797"]
path1 = structure_path + "/" + uid[0] + ".json"
path2 = structure_path + "/" + uid[1] + ".json"


In [6]:
# extract the cell vectors from structure file
cell1 = get_cell_from_structure_file(path1)
cell2 = get_cell_from_structure_file(path2)


In [7]:
# Option step align the cells
aligned_cells = allign_along_10([cell1, cell2])
cel1 = aligned_cells[0]
cel2 = aligned_cells[1]
print("=== \n cel1:\n {}".format(cell1))
print("\n cel2:\n {}\n===".format(cell2))


=== 
 cel1:
 [[ 3.59357271e+00 -7.05955135e-17]
 [ 8.16101380e-17  3.59357271e+00]]

 cel2:
 [[ 4.33626074e+00 -2.16813037e+00]
 [-6.32390570e-17  3.75531195e+00]]
===


## Build a solver and run the experiment

### Annealing

In [8]:
# Select a solver and modify the solver parameters

print("\n\nAnnealingSolver:")
solver1 = AnnealingSolver()
strain_boundary = [[-0.3, 0.3], [-0.3, 0.3]]
solver1.nr_epochs = 11

# Calculate super cell
super_cell1 = solver1.solve(cell1, cell2)



AnnealingSolver:
temp:4|epoch:0|change:1425939539414.5864|value:502680642654.4413
temp:4|epoch:5|change:-459102919420.77045|value:43577723233.67085
temp:4|epoch:10|change:805259217194.8792|value:43577723233.67085
temp:0.0975609756097561|epoch:0|change:353390138919.54065|value:43577723233.67085
temp:0.0975609756097561|epoch:5|change:244881814535.7292|value:43577723233.67085
temp:0.0975609756097561|epoch:10|change:3672011817317.6094|value:43577723233.67085
temp:0.04938271604938272|epoch:0|change:25491536648056.875|value:43577723233.67085
temp:0.04938271604938272|epoch:5|change:55058611662298.6|value:3629827651.9880958
temp:0.04938271604938272|epoch:10|change:3690414980092.4365|value:3629827651.9880958
temp:0.03305785123966942|epoch:0|change:3150965193792.9565|value:3629827651.9880958
temp:0.03305785123966942|epoch:5|change:11303009375640.34|value:3629827651.9880958
temp:0.03305785123966942|epoch:10|change:14868821934327.422|value:3629827651.9880958
temp:0.02484472049689441|epoch:0|chan

temp:0.0029390154298310073|epoch:10|change:75283544174.82532|value:11148939.185309269
temp:0.002855103497501785|epoch:0|change:9566062566990.418|value:11148939.185309269
temp:0.002855103497501785|epoch:5|change:2281578342362.4395|value:11148939.185309269
temp:0.002855103497501785|epoch:10|change:2793962609231.633|value:11148939.185309269
temp:0.0027758501040943797|epoch:0|change:9984626877934.785|value:11148939.185309269
temp:0.0027758501040943797|epoch:5|change:99943866401.55685|value:11148939.185309269
temp:0.0027758501040943797|epoch:10|change:149421089885.7013|value:11148939.185309269
temp:0.0027008777852802167|epoch:0|change:35318669959.069305|value:11148939.185309269
temp:0.0027008777852802167|epoch:5|change:6295281504.074842|value:11148939.185309269
temp:0.0027008777852802167|epoch:10|change:2837007726344.5376|value:11148939.185309269
temp:0.002629848783694938|epoch:0|change:79585841758.22641|value:11148939.185309269
temp:0.002629848783694938|epoch:5|change:5642886643720.455|val

temp:0.001470047776552738|epoch:10|change:162240931709.68195|value:11148939.185309269
temp:0.0014487504527345167|epoch:0|change:1648908565108.487|value:11148939.185309269
temp:0.0014487504527345167|epoch:5|change:11197299958819.379|value:11148939.185309269
temp:0.0014487504527345167|epoch:10|change:260708401200.09525|value:11148939.185309269
temp:0.0014280614066404858|epoch:0|change:207078588990.07678|value:11148939.185309269
temp:0.0014280614066404858|epoch:5|change:311680319309.7382|value:11148939.185309269
temp:0.0014280614066404858|epoch:10|change:3146168987331.4126|value:11148939.185309269
temp:0.001407954945441746|epoch:0|change:3995824420257.1353|value:11148939.185309269
temp:0.001407954945441746|epoch:5|change:286793574918.93256|value:11148939.185309269
temp:0.001407954945441746|epoch:10|change:2158884555.2957516|value:11148939.185309269
temp:0.0013884068031933359|epoch:0|change:57529606866853.5|value:11148939.185309269
temp:0.0013884068031933359|epoch:5|change:1726195120468.60

temp:0.0009898539965355103|epoch:5|change:850005450648.4473|value:11148939.185309269
temp:0.0009898539965355103|epoch:10|change:1053963958813.1298|value:11148939.185309269
temp:0.0009801519235481493|epoch:0|change:1629773035130.163|value:11148939.185309269
temp:0.0009801519235481493|epoch:5|change:411151399077.8745|value:11148939.185309269
temp:0.0009801519235481493|epoch:10|change:2814150651667.248|value:11148939.185309269
temp:0.0009706381946129574|epoch:0|change:677681426713.2716|value:11148939.185309269
temp:0.0009706381946129574|epoch:5|change:1902212293765.7507|value:11148939.185309269
temp:0.0009706381946129574|epoch:10|change:566715726653.2374|value:11148939.185309269
temp:0.0009613073780341257|epoch:0|change:3966662079346.886|value:11148939.185309269
temp:0.0009613073780341257|epoch:5|change:3336278629937.3403|value:11148939.185309269
temp:0.0009613073780341257|epoch:10|change:6562802211279.605|value:11148939.185309269
temp:0.0009521542489883355|epoch:0|change:2852166153196.35

temp:0.0007406035919274203|epoch:5|change:19658288213287.21|value:11148939.185309269
temp:0.0007406035919274203|epoch:10|change:242138198243.38263|value:11148939.185309269
temp:0.0007351589781290198|epoch:0|change:236466426732.02805|value:11148939.185309269
temp:0.0007351589781290198|epoch:5|change:110239442512.12817|value:11148939.185309269
temp:0.0007351589781290198|epoch:10|change:747469905.6916317|value:11148939.185309269
temp:0.0007297938332421085|epoch:0|change:73918600780.47011|value:11148939.185309269
temp:0.0007297938332421085|epoch:5|change:7151032048375.792|value:11148939.185309269
temp:0.0007297938332421085|epoch:10|change:1993549298166.1943|value:11148939.185309269
temp:0.0007245064299945656|epoch:0|change:224105175217.81345|value:11148939.185309269
temp:0.0007245064299945656|epoch:5|change:1990832663299.354|value:11148939.185309269
temp:0.0007245064299945656|epoch:10|change:43071162882.19917|value:11148939.185309269
temp:0.0007192950908110046|epoch:0|change:10694290632761

...

temp:0.0002724610040188|epoch:5|change:141023613090.39572|value:11148939.185309269
temp:0.0002724610040188|epoch:10|change:20598550196480.09|value:11148939.185309269
temp:0.00027172067115005787|epoch:0|change:868035819883.763|value:11148939.185309269
temp:0.00027172067115005787|epoch:5|change:8608211516210.632|value:11148939.185309269
temp:0.00027172067115005787|epoch:10|change:9983256136961.797|value:11148939.185309269
temp:0.00027098435065374984|epoch:0|change:333667657575.125|value:11148939.185309269
temp:0.00027098435065374984|epoch:5|change:670390396571.419|value:11148939.185309269
temp:0.00027098435065374984|epoch:10|change:1000010333997975.2|value:11148939.185309269
temp:0.00027025200999932446|epoch:0|change:3427110088200.559|value:11148939.185309269
temp:0.00027025200999932446|epoch:5|change:5013413917933.412|value:11148939.185309269
temp:0.00027025200999932446|epoch:10|change:1879266472.380062|value:11148939.185309269
temp:0.0002695236170069403|epoch:0|change:212837380822.5641

temp:0.0002506108639809535|epoch:10|change:275752496061.74054|value:11148939.185309269
temp:0.0002499843759765014|epoch:0|change:77395403806.3528|value:11148939.185309269
temp:0.0002499843759765014|epoch:5|change:1550215664.664935|value:11148939.185309269
temp:0.0002499843759765014|epoch:10|change:1148346572523.9111|value:11148939.185309269
temp:0.00024936101240571033|epoch:0|change:111165917283.531|value:11148939.185309269
temp:0.00024936101240571033|epoch:5|change:11474559338919.012|value:11148939.185309269
temp:0.00024936101240571033|epoch:10|change:9126037176829.613|value:11148939.185309269
temp:0.0002487407499533611|epoch:0|change:10901439209497.682|value:11148939.185309269
temp:0.0002487407499533611|epoch:5|change:1515902320380.4846|value:11148939.185309269
temp:0.0002487407499533611|epoch:10|change:801577956784.6628|value:11148939.185309269
temp:0.0002481235655356367|epoch:0|change:170201990570.61047|value:11148939.185309269
temp:0.0002481235655356367|epoch:5|change:444846054453

temp:0.0002314680863375962|epoch:10|change:5144434812126.276|value:11148939.185309269
temp:0.00023093354887131226|epoch:0|change:161780588226.27856|value:11148939.185309269
temp:0.00023093354887131226|epoch:5|change:1266683252738.0247|value:11148939.185309269
temp:0.00023093354887131226|epoch:10|change:13377102254159.889|value:11148939.185309269
temp:0.00023040147456943722|epoch:0|change:2412808300267.253|value:11148939.185309269
temp:0.00023040147456943722|epoch:5|change:12497667722295.156|value:11148939.185309269
temp:0.00023040147456943722|epoch:10|change:15765982331909.826|value:11148939.185309269
temp:0.00022987184644560657|epoch:0|change:41542781670.619644|value:11148939.185309269
temp:0.00022987184644560657|epoch:5|change:983527924551.0098|value:11148939.185309269
temp:0.00022987184644560657|epoch:10|change:2374558105512.781|value:11148939.185309269
temp:0.00022934464766928502|epoch:0|change:19655587478.051083|value:11148939.185309269
temp:0.00022934464766928502|epoch:5|change:2

temp:0.00021504220203214875|epoch:10|change:5249536269.022265|value:11148939.185309269
temp:0.00021458076283461183|epoch:0|change:5203965662939.368|value:11148939.185309269
temp:0.00021458076283461183|epoch:5|change:1006001688145483.9|value:11148939.185309269
temp:0.00021458076283461183|epoch:10|change:89636898713.01947|value:11148939.185309269
temp:0.00021412129971628925|epoch:0|change:1000013714516666.5|value:11148939.185309269
temp:0.00021412129971628925|epoch:5|change:6861627544514.111|value:11148939.185309269
temp:0.00021412129971628925|epoch:10|change:28863155350139.484|value:11148939.185309269
temp:0.00021366380001068314|epoch:0|change:160742607511.00162|value:11148939.185309269
temp:0.00021366380001068314|epoch:5|change:3152961351492.6475|value:11148939.185309269
temp:0.00021366380001068314|epoch:10|change:6282961714257.87|value:11148939.185309269
temp:0.0002132082511593198|epoch:0|change:7443740840.49964|value:11148939.185309269
temp:0.0002132082511593198|epoch:5|change:266388

temp:0.0002007931328748556|epoch:5|change:798833773982.7332|value:11148939.185309269
temp:0.0002007931328748556|epoch:10|change:646704514553.3342|value:11148939.185309269
temp:0.00020039076198587235|epoch:0|change:36223010531756.09|value:11148939.185309269
temp:0.00020039076198587235|epoch:5|change:37195297919911.37|value:11148939.185309269
temp:0.00020039076198587235|epoch:10|change:1948183927608.039|value:11148939.185309269


In [9]:
print(super_cell1.description_txt())


=== 
 cell :
[[ 1.43742908e+01 -7.18714542e+00]
 [ 8.16101380e-17  3.59357271e+00]]

cell1 :
[[ 3.59357271e+00 -7.05955135e-17]
 [ 8.16101380e-17  3.59357271e+00]]

cell2 :
[[ 4.33626074e+00 -2.16813037e+00]
 [-6.32390570e-17  3.75531195e+00]]

ta :
[[ 4 -2]
 [ 0  1]]

strain_tb :
[[3.31490464e+00 4.44089210e-16]
 [3.27760556e-17 9.56930544e-01]]

det_ta :
4.0

strain :
[[1.10496821e+00 4.44089210e-16]
 [1.09253519e-17 9.56930544e-01]]

tb :
[[3. 0.]
 [0. 1.]]

diagonal_strain :
[[1.09279155 0.        ]
 [0.         0.97822826]]

diagonal_strain_tb: :
[[3.03342814e+00 4.06380530e-16]
 [3.35055292e-17 9.78228272e-01]]




### Genetic

In [10]:
print("\n\nGeneticSolver:")
solver2 = GeneticSolver()
strain_boundary = [[-0.3, 0.3], [-0.3, 0.3]]
solver2.model_par["subjects_in_cell"] = 2
solver2.model_par["pins"] = 4

# Calculate super cell
super_cell2 = solver2.solve(cell1, cell2)



GeneticSolver:
Epochs 0/10
perf [-204962641576.6289, -33551780228.610783, -3570668934083.7705, -94549471544842.6, -1857755539001.9194, -4381005937978609.0, -1897098471005208.2, -277307949789237.8, -1101333400756726.0, -152752493894165.12, -1540638560993.6743, -22458132351299.066, -22926916126277.96, -43007013212806.46, -69921738275509.89, -73321057537308.1, -17728168984.118, -23061140762.616596, -959667861030250.1, -376974071403.46747, -57507095987008.29, -425404555953.74603, -1130166392170.3152, -32859305356962.75, -16040924856274.725, -337971044298.7111, -564112884991.3291, -35765593723.77283, -121681125873137.58, -4891136591601.403, -2875064417600.458, -1791974959106.1118, -124784695841.03577, -677337192.8769403, -2025394873.5636053, -101492669284419.02, -278147094.62508005, -759138689132.199, -187146951672236.06, -135564637455593.16, -2245310004435627.5, -30201398311911.055, -179754287950205.3, -325833968829178.7, -1884169842490134.2, -2.9055135937565944e+16, -12949020887657.676,

In [11]:
print(super_cell2.description_txt())


=== 
 cell :
[[-1.63220276e-16 -7.18714542e+00]
 [ 2.15614363e+01 -1.07807181e+01]]

cell1 :
[[ 3.59357271e+00 -7.05955135e-17]
 [ 8.16101380e-17  3.59357271e+00]]

cell2 :
[[ 4.33626074e+00 -2.16813037e+00]
 [-6.32390570e-17  3.75531195e+00]]

ta :
[[ 0 -2]
 [ 6 -3]]

strain_tb :
[[-6.55521113e-17 -1.91386109e+00]
 [ 4.97235697e+00  8.88178420e-16]]

det_ta :
12.0

strain :
[[ 9.56930544e-01 -1.31104223e-17]
 [-4.44089210e-16  9.94471393e-01]]

tb :
[[ 0. -2.]
 [ 5.  0.]]

diagonal_strain :
[[0.96512194 0.        ]
 [0.         0.9946819 ]]

diagonal_strain_tb: :
[[-6.79210661e-17 -1.98302515e+00]
 [ 4.99894184e+00  8.92927095e-16]]




### Mechanic 

In [12]:
print("\n\nMechanicSolver:")
solver3 = MechanicSolver()
solver3.exploring_range = 50
solver3.tolerance = 0.4
# Calculate super cell
super_cell3 = solver3.solve(cell1, cell2)



MechanicSolver:


In [13]:
print(super_cell3.description_txt())


=== 
 cell :
[[ 1.41191027e-16 -1.07807181e+01]
 [-7.18714542e+00 -3.59357271e+00]]

cell1 :
[[ 3.59357271e+00 -7.05955135e-17]
 [ 8.16101380e-17  3.59357271e+00]]

cell2 :
[[ 4.33626074e+00 -2.16813037e+00]
 [-6.32390570e-17  3.75531195e+00]]

ta :
[[ 1.07419961e-16 -3.00000000e+00]
 [-2.00000000e+00 -1.00000000e+00]]

strain_tb :
[[-9.30643497e-18 -2.87079163e+00]
 [-1.65745232e+00 -1.91386109e+00]]

det_ta :
-6.0

strain :
[[9.56930544e-01 4.65321748e-18]
 [8.54695888e-02 8.28726161e-01]]

tb :
[[ 0. -3.]
 [-2. -2.]]

diagonal_strain :
[[0.96095367 0.        ]
 [0.         0.90713593]]

diagonal_strain_tb: :
[[-9.68458235e-18 -2.98744020e+00]
 [-1.82712675e+00 -2.10978424e+00]]


